In [ ]:
import os
import os.path as osp
import pandas as pd
from glob import glob

slice = '0_*'

In [ ]:
data_dir = osp.abspath('../collected_data')

ann_dir = osp.join(data_dir, 'intermediate', 'head_noun_extraction')
input_files = glob(osp.join(ann_dir, slice, 'collected_annotations_clean_human_*.csv'))

processed_dfs = [pd.read_csv(f, index_col=0) for f in input_files]
processed_df = pd.concat(processed_dfs).sort_index()
processed_df.comments = processed_df.comments.fillna('')
processed_df.head()

In [ ]:
user_dir = '../generated_data/experiment_slices'
user_files = glob(osp.join(user_dir, f'argilla_users_{slice}.csv'))

user_data = pd.concat([pd.read_csv(f, index_col=0) for f in user_files])
valid_map = user_data.set_index('username').valid.to_dict()
user_data.head()

In [ ]:
data_dir

In [ ]:
raw_ann_dir = osp.join(data_dir, 'raw')
raw_ann_files = glob(osp.join(raw_ann_dir, slice, 'collected_annotations.csv'))

collected_df = pd.concat([pd.read_csv(f, index_col=0) for f in raw_ann_files])
collected_df = collected_df[[c for c in collected_df.columns if c not in ['raw_annotation', 'user_id']]]

collected_df.head()


In [ ]:
assert len(collected_df) == len(processed_df)

In [ ]:
merged_df = pd.merge(processed_df, collected_df, left_on='item_identifyer', right_on='item_identifyer')
merged_df['valid'] = merged_df.user_name.map(valid_map)

out_name = 'full_processed_collected_data.csv'
out_path = osp.join(data_dir, out_name)
merged_df.to_csv(out_path)

In [ ]:
merged_df.head()

In [ ]:
valid_df = merged_df[merged_df.valid]

out_name = 'valid_processed_collected_data.csv'
out_path = osp.join(data_dir, out_name)

valid_df.to_csv(out_path)

In [ ]:
valid_df.head()